In [1]:
import os
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action="ignore")
    
all_data_path = []
for i in range(1,5):
    path = f"../NIKL_DIALOGUE_2020_PCM_v1.3_part{i}/pcm"
    folder_names = os.listdir(path)
    folder_names = [folder_name for folder_name in folder_names]
    #print(folder_names)

    data_lst = []
    for folder_name in folder_names:
        file_names = os.listdir('{}/{}'.format(path, folder_name))
        file_names = [file_name for file_name in file_names]
    #     print(file_names)

        for file_name in file_names:
            if file_name in file_names:
                file_path = '{}/{}/{}'.format(path, folder_name, file_name)
                data_lst.append(file_path)
    all_data_path.append(data_lst)

In [2]:
df =  pd.DataFrame(all_data_path[0])

In [3]:
df [0][0]

'../NIKL_DIALOGUE_2020_PCM_v1.3_part1/pcm/SDRW2000000408/SDRW2000000408.1.1.287.pcm'

In [4]:
import struct
import pathlib
import os

def make_wav_format(pcm_data: bytes, ch: int) -> bytes:
    waves = []
    waves.append(struct.pack('<4s', b'RIFF'))
    waves.append(struct.pack('I', 1))
    waves.append(struct.pack('4s', b'WAVE'))
    waves.append(struct.pack('4s', b'fmt '))
    waves.append(struct.pack('I', 16))

    if ch == 2:
        waves.append(struct.pack('HHIIHH', 1, 2, 16000, 64000, 4, 16))
    else:
        waves.append(struct.pack('HHIIHH', 1, 1, 16000, 32000, 2, 16))
    waves.append(struct.pack('<4s', b'data'))
    waves.append(struct.pack('I', len(pcm_data)))
    waves.append(pcm_data)
    waves[1] = struct.pack('I', sum(len(w) for w in waves[2:]))
    return b''.join(waves)

# 디렉토리 생성
output_dir = 'wav_part1'
os.makedirs(output_dir, exist_ok=True)

count = 0
for i in range(len(df.index)):
    pcm_bytes = pathlib.Path(df.iloc[i, 0]).read_bytes()
    wav_bytes = make_wav_format(pcm_bytes, 1)

    # WAV 파일 경로 생성
    file_name = os.path.basename(df.iloc[i, 0])
    file_name_without_extension = os.path.splitext(file_name)[0]
    wav_file_name = file_name_without_extension + '.wav'
    wav_file_path = os.path.join(output_dir, wav_file_name)

    # WAV 파일 저장
    with open(wav_file_path, 'wb') as file:
        file.write(wav_bytes)